# OAS->requests

In [ ]:
#| default_exp core.oas_to_requests

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import requests
from pprint import pprint
import json
import yaml
import copy
import re
from typing import Callable, Optional

## OAS inspection

This implementation focuses on DigiTraffic with its OpenAPI specification file.

In [ ]:
import requests
import json
import os

BASE_URL = "https://tie.digitraffic.fi"

# Download a file from a URL
def download_file(url, dest_file):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(dest_file, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

# Define the OpenAPI file path
local_file = "openapi.json"

# Download the OpenAPI JSON file if it doesn't exist
openapi_url = f"{BASE_URL}/swagger/openapi.json"  # Update with the correct path
if not os.path.exists(local_file):
    download_file(openapi_url, local_file)

# Load and parse the OpenAPI JSON file
with open(local_file, 'r') as f:
    spec_dict = json.load(f)
oas = spec_dict

### Deep reference extraction

In order to generate tool schemas, we need to resolve and flatten the references to `components`.

In [ ]:
#| export
MIME_TYPES = {
    "text": [
        "plain",
        "html",
        "css",
        "javascript",
        "markdown",
        "xml",
        "csv",
        "tab-separated-values",
        "vcard"
    ],
    "application": [
        "json",
        "xml",
        "javascript",
        "x-www-form-urlencoded",
        "multipart-form-data",
        "pdf",
        "zip",
        "gzip",
        "vnd.api+json",
        "sql",
        "octet-stream",
        "ld+json",
        "vnd.ms-excel",
        "vnd.openxmlformats-officedocument.spreadsheetml.sheet",
        "vnd.ms-word",
        "vnd.openxmlformats-officedocument.wordprocessingml.document",
        "vnd.ms-powerpoint",
        "vnd.openxmlformats-officedocument.presentationml.presentation",
        "x-tar",
        "x-7z-compressed",
        "vnd.android.package-archive",
        "x-rar-compressed",
        "x-bzip",
        "x-bzip2",
        "x-sh",
        "x-java-archive",
        "x-httpd-php",
        "x-pkcs12",
        "x-pkcs7-certificates",
        "x-pkcs7-mime",
        "x-pkcs7-signature"
    ],
    "image": [
        "png",
        "jpeg",
        "gif",
        "webp",
        "svg+xml",
        "bmp",
        "tiff",
        "x-icon"
    ],
    "audio": [
        "mpeg",
        "ogg",
        "wav",
        "webm",
        "aac",
        "midi"
    ],
    "video": [
        "mp4",
        "ogg",
        "webm",
        "x-msvideo",
        "quicktime"
    ],
    "font": [
        "ttf",
        "otf",
        "woff",
        "woff2"
    ]
}

def retrieve_ref_parts(refs: str):
    """Retrieve the parts of a reference string"""
    # Split the reference string into raw parts and initialize the parts list
    raw_parts = refs.split("/")
    parts = []

    i = 0
    while i < len(raw_parts) - 1:
        # Extract consecutive parts
        first_part = raw_parts[i]
        second_part = raw_parts[i + 1]

        # Check if the parts are valid MIME types
        if first_part in MIME_TYPES and second_part in MIME_TYPES[first_part]:
            # Combine the MIME types
            parts.append(f"{first_part}/{second_part}")
            i += 2
        else:
            parts.append(first_part)
            i += 1

    # Add the last part if it is not a MIME type
    if raw_parts[-2] not in MIME_TYPES or raw_parts[-1] not in MIME_TYPES[raw_parts[-2]]:
        parts.append(raw_parts[-1])

    return parts

In [ ]:
#| export
def extract_refs(
        oas: dict  # The OpenAPI schema
    ) -> dict:  # The extracted references (flattened)
    refs = copy.deepcopy(oas)
    refs_list = set()
    refs_locations = {}
    refs_dependencies = {}

    # Traverse the components section of the spec
    for section, items in refs["components"].items():
        for item in items:
            refs_list.add(f"components/{section}/{item}")
            refs_locations[f"components/{section}/{item}"] = []
            refs_dependencies[f"components/{section}/{item}"] = set()
    
    # Initialize the clean_refs set
    clean_refs = refs_list.copy()

    # Traverse the spec and extract the references
    def traverse_location(obj, path=""):
        for key, value in obj.items():
            if key == "$ref":
                # Determine the root of the reference and remove it from the clean_refs set
                ref_root = "/".join(path.split("/")[:3])
                clean_refs.discard(ref_root)

                # Extract the sub reference and add the current path to the list of locations
                sub_ref = value[2:]
                refs_locations[sub_ref].append(path)

                # Add the sub reference to the dependencies of the current reference
                refs_dependencies[ref_root].add(sub_ref)

            elif isinstance(value, dict):
                # Recursively traverse the object
                traverse_location(value, f"{path}/{key}")

    traverse_location(refs["components"], "components")

    # Attach the reference objects to the locations
    def attach_clean_refs():
        for ref in clean_refs:
            # Extract the reference object
            ref_obj = refs
            ref_paths = retrieve_ref_parts(ref)
            for part in ref_paths:
                ref_obj = ref_obj[part]

            # Extract the locations where the reference is used
            locations = refs_locations[ref]

            # Attach the reference object to the locations
            for location in locations:
                location_parts = retrieve_ref_parts(location)

                obj = refs
                prev = None
                for part in location_parts:
                    prev = obj
                    obj = obj[part]

                prev[location_parts[-1]] = ref_obj

            # Remove the reference from the dependencies
            for dependency in refs_dependencies:
                refs_dependencies[dependency].discard(ref)

    # Check if there are any clean references
    def check_clean_refs():
        clean_refs = set()
        for ref, dependencies in refs_dependencies.items():
            if len(dependencies) == 0:
                clean_refs.add(ref)
        return clean_refs
    
    # Iterate until all references are attached or no progress is made
    prev_nof_clean = None
    while len(clean_refs) < len(refs_list) and prev_nof_clean != len(clean_refs):
        prev_nof_clean = len(clean_refs)
        attach_clean_refs()
        clean_refs = check_clean_refs()

    # Flatten the references
    flatten_refs = {}
    for section, items in refs["components"].items():
        for item in items:
            flatten_refs[f"#/components/{section}/{item}"] = refs["components"][section][item]

    return flatten_refs

Examples from DigiTraffic components:

In [ ]:
refs = extract_refs(oas)
for i, (ref, obj) in enumerate(list(refs.items())[:1]):
    print(f"Reference: {ref}", list(obj.keys()))
    for j, (k,v) in enumerate(obj.items()):
        print(f"[{i}{j}]", k, json.dumps(v, indent=4) if k=='properties' else v)

Reference: #/components/schemas/WeatherSensorValueHistoryDto ['type', 'properties']
[00] type object
[01] properties {
    "roadStationId": {
        "type": "integer",
        "description": "Road station id",
        "format": "int64"
    },
    "sensorId": {
        "type": "integer",
        "description": "Sensor id",
        "format": "int64"
    },
    "sensorValue": {
        "type": "number",
        "description": "Sensor value",
        "format": "double"
    },
    "measured": {
        "type": "string",
        "format": "date-time",
        "writeOnly": true
    },
    "reliability": {
        "type": "string",
        "description": "Measurement reliability information",
        "enum": [
            "OK",
            "SUSPICIOUS",
            "FAULTY",
            "UNKNOWN"
        ]
    },
    "measuredTime": {
        "type": "string",
        "description": "Value's measured date time",
        "format": "date-time"
    }
}


In [ ]:
show_doc(extract_refs)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/oas_to_requests.py#L123){target="_blank" style="float:right; font-size:smaller"}

### extract_refs

>      extract_refs (oas:dict)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| oas | dict | The OpenAPI schema |
| **Returns** | **dict** | **The extracted references (flattened)** |

### OAS schema to GPT-compatible schema

GPT currently recognizes only a limited number of descriptors when defining toolbox schema. Some of these descriptors (fields) can be directly transferred from OAS schema to toolbox, but many existing OAS schema fields will not be recognized by GPT and can cause errors. Therefore, transformation from OAS schemas to GPT-compatible schemas is necessary.

GPT currently recognizes these fields:

1. `type`  
Specifies the data type of the value. Common types include:  
  - `string` – A text string.  
  - `number` – A numeric value (can be integer or floating point).  
  - `integer` – A whole number.  
  - `boolean` – A true/false value.  
  - `array` – A list of items (you can define the type of items in the array as well).  
  - `object` – A JSON object (with properties, which can be further defined with their own types).  
  - `null` – A special type to represent a null or absent value.  
  - `any` – Allows any type, typically used for flexible inputs or outputs.  
2. `default`: Provides a default value for the field if the user doesn't supply one. It can be any valid type based on the expected schema.  
3. `enum`: Specifies a list of acceptable values for a field. It restricts the input to one of the predefined values in the array.  
4. `properties`: Used for objects, this defines the subfields of an object and their respective types.  
5. `items`: Defines the type of items in an array. For example, you can specify that an array contains only strings or integers.  
6. `minLength`, `maxLength`: Specifies minimum and maximum lengths for `string` parameters.  
7. `minItems`, `maxItems`: Specifies mininum and maximum number of items for `array` parameters.  
8. `pattern`: Specifies a regular expression that the string must match for `string` parameters.  
9. `required`: A list of required fields for an `object`. Specifies that certain fields within an `object` must be provided.  
10. `additionalProperties`: Specifies whether additional properties are allowed in an `object`. If set to `false`, no properties outside of those defined in properties will be accepted.

As such, we can extract corresponding fields from OAS schema, and converts all additional fields into parameter description.

In [ ]:
#| export
# Directly transferable properties from OAS to GPT-compatible schema
TRANSFERABLE_TYPES = [
    "type", "description", "default", "enum", "pattern", "additionalProperties",
    "minLength", "maxLength", "minItems", "maxItems"
]

# Function to transform OAS schema to GPT-compatible schema
def transform_property(
        prop: dict,  # The property to transform
        flatten_refs: dict = {}  # The flattened references
    ) -> tuple[dict, bool]:  # The transformed property and whether it is a required property

    # If the property is a schema, flatten it
    if "schema" in prop:
        prop = copy.deepcopy(prop)
        prop.update(prop["schema"])
        prop.pop("schema")

    # Extract the required field
    required = prop.get("required", False)
    
    # If the property is a reference, return it as is
    if "$ref" in prop:
        if prop["$ref"] in flatten_refs:
            ref_prop, _ = transform_property(flatten_refs[prop["$ref"]], flatten_refs)
            return ref_prop, required
        else:
            # If the reference is not found, return the reference as is
            return prop, required 
    
    # If the property is an object, transform it
    new_prop = {}
    additionals = {}

    # If required is a list, it is directly transferable to GPT-compatible schema
    if isinstance(required, list): 
        new_prop["required"] = required
        required = True

    for key, value in prop.items():
        if key in TRANSFERABLE_TYPES:
            new_prop[key] = value
        elif key == "items":
            # Handle array items recursively
            item_prop, _ = transform_property(value, flatten_refs)
            new_prop[key] = item_prop
        elif key == "properties":
            # Handle nested properties recursively
            new_prop[key] = {}
            new_prop["required"] = [] if "required" not in new_prop else new_prop["required"]
            for sub_key, sub_value in value.items():
                sub_prop, sub_required = transform_property(sub_value, flatten_refs)
                new_prop[key][sub_key] = sub_prop
                if sub_required:
                    new_prop["required"].append(sub_key)
        elif key == "required":
            # Skip required field since it is handled in the properties section
            continue
        else:
            # Collect unrecognized fields in additionals dictionary
            additionals[key] = value

    # Add the additionals dictionary if it is not empty
    if len(additionals) > 0:
        additional_info = "; ".join([f"{k.capitalize()}: {v}" for k, v in additionals.items()])
        if "description" in new_prop:
            new_prop["description"] += f" ({additional_info})"
        else:
            new_prop["description"] = f"({additional_info})"

    # Remove None values and return the transformed property
    return {k: v for k, v in new_prop.items() if v is not None}, required


In [ ]:
show_doc(transform_property)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/oas_to_requests.py#L221){target="_blank" style="float:right; font-size:smaller"}

### transform_property

>      transform_property (prop:dict, flatten_refs:dict={})

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| prop | dict |  | The property to transform |
| flatten_refs | dict | {} | The flattened references |
| **Returns** | **tuple** |  | **The transformed property and whether it is a required property** |

Test usage with complex parameters from DigiTraffic endpoints:

In [ ]:
parameters = [
    {
        "name": "lastUpdated",
        "in": "query",
        "description": "If parameter is given result will only contain update status.",
        "required": False,
            "schema": {
                "type": "boolean",
                "default": False
            }
    },
    {
        "name": "roadNumber",
        "in": "query",
        "description": "Road number",
        "required": False,
        "schema": {
            "type": "integer",
            "format": "int32"
        }
    },
    {
        "name": "xMin",
        "in": "query",
        "description": "Minimum x coordinate (longitude) Coordinates are in WGS84 format in decimal degrees. Values between 19.0 and 32.0.",
        "required": False,
        "schema": {
            "maximum": 32,
            "exclusiveMaximum": False,
            "minimum": 19,
            "exclusiveMinimum": False,
            "type": "number",
            "format": "double",
            "default": 19
        }
    },
    {
        "name": "yMin",
        "in": "query",
        "description": "Minimum y coordinate (latitude). Coordinates are in WGS84 format in decimal degrees. Values between 59.0 and 72.0.",
        "required": False,
        "schema": {
            "maximum": 72,
            "exclusiveMaximum": False,
            "minimum": 59,
            "exclusiveMinimum": False,
            "type": "number",
            "format": "double",
            "default": 59
        }
    },
    {
        "name": "xMax",
        "in": "query",
        "description": "Maximum x coordinate (longitude). Coordinates are in WGS84 format in decimal degrees. Values between 19.0 and 32.0.",
        "required": False,
        "schema": {
            "maximum": 32,
            "exclusiveMaximum": False,
            "minimum": 19,
            "exclusiveMinimum": False,
            "type": "number",
            "format": "double",
            "default": 32
        }
    },
    {
        "name": "yMax",
        "in": "query",
        "description": "Maximum y coordinate (latitude). Coordinates are in WGS84 format in decimal degrees. Values between 59.0 and 72.0.",
        "required": False,
        "schema": {
            "maximum": 72,
            "exclusiveMaximum": False,
            "minimum": 59,
            "exclusiveMinimum": False,
            "type": "number",
            "format": "double",
            "default": 72
        }
    }
]

for param in parameters:
    param, required = transform_property(param)
    print(json.dumps(param, indent=2))
    print(f"Required: {required}\n")

{
  "description": "If parameter is given result will only contain update status. (Name: lastUpdated; In: query)",
  "type": "boolean",
  "default": false
}
Required: False

{
  "description": "Road number (Name: roadNumber; In: query; Format: int32)",
  "type": "integer"
}
Required: False

{
  "description": "Minimum x coordinate (longitude) Coordinates are in WGS84 format in decimal degrees. Values between 19.0 and 32.0. (Name: xMin; In: query; Maximum: 32; Exclusivemaximum: False; Minimum: 19; Exclusiveminimum: False; Format: double)",
  "type": "number",
  "default": 19
}
Required: False

{
  "description": "Minimum y coordinate (latitude). Coordinates are in WGS84 format in decimal degrees. Values between 59.0 and 72.0. (Name: yMin; In: query; Maximum: 72; Exclusivemaximum: False; Minimum: 59; Exclusiveminimum: False; Format: double)",
  "type": "number",
  "default": 59
}
Required: False

{
  "description": "Maximum x coordinate (longitude). Coordinates are in WGS84 format in dec

In [ ]:
#| hide
# Sample OAS schema with nested schema properties

flatten_refs = {
    "#/components/schemas/dimensions": {
        "type": "object",
        "properties": {
            "width": {
                "type": "number",
                "minimum": 0,
                "description": "Width in centimeters.",
                "required": True
            },
            "height": {
                "type": "number",
                "minimum": 0,
                "description": "Height in centimeters.",
                "required": True
            },
            "depth": {
                "type": "number",
                "minimum": 0,
                "description": "Depth in centimeters.",
                "required": True
            }
        },
        "description": "Dimensions of the product."
    }
}
        
nested_param = {
    "type": "object",
    "properties": {
        "product": {
            "type": "object",
            "schema": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "string",
                        "description": "Unique identifier for the product."
                    },
                    "details": {
                        "type": "object",
                        "schema": {
                            "type": "object",
                            "properties": {
                                "weight": {
                                    "type": "number",
                                    "minimum": 0,
                                    "description": "Weight of the product in kilograms."
                                },
                                "dimensions": {
                                    "schema":
                                        {
                                        "$ref": "#/components/schemas/dimensions"
                                        }
                                }
                            }
                        },
                        "description": "Detailed specifications of the product."
                    }
                }
            },
            "description": "Product information."
        },
        "category": {
            "type": "string",
            "description": "Category of the product."
        }
    },
    "required": ["product", "category"]
}

transformed_param, _ = transform_property(nested_param, flatten_refs)
assert transformed_param == {
  "required": [
    "product",
    "category"
  ],
  "type": "object",
  "properties": {
    "product": {
      "type": "object",
      "description": "Product information.",
      "properties": {
        "id": {
          "type": "string",
          "description": "Unique identifier for the product."
        },
        "details": {
          "type": "object",
          "description": "Detailed specifications of the product.",
          "properties": {
            "weight": {
              "type": "number",
              "description": "Weight of the product in kilograms. (Minimum: 0)"
            },
            "dimensions": {
              "type": "object",
              "description": "Dimensions of the product.",
              "properties": {
                "width": {
                  "type": "number",
                  "description": "Width in centimeters. (Minimum: 0)"
                },
                "height": {
                  "type": "number",
                  "description": "Height in centimeters. (Minimum: 0)"
                },
                "depth": {
                  "type": "number",
                  "description": "Depth in centimeters. (Minimum: 0)"
                }
              },
              "required": [
                "width",
                "height",
                "depth"
              ]
            }
          },
          "required": []
        }
      },
      "required": []
    },
    "category": {
      "type": "string",
      "description": "Category of the product."
    }
  }
}

    

## Executing requests with GPT

### Auxiliary function to generate requests

In [ ]:
#| export
def generate_request(
    function_name: str,  # The name of the function
    url: str,  # The URL of the request
    method: str,  # The method of the request
    path: dict = {},  # The path parameters of the request
    query: dict = {},  # The query parameters of the request
    body: dict = {},  # The body of the request
    accepted_queries: list = [],  # The accepted queries of the request
    **kwargs  # Additional parameters
) -> dict:  # The response of the request
    """Generate a request from the function name and parameters."""
    # Extract the URL and method from the toolbox if not provided
    if url is None or method is None:
        raise ValueError("URL and method must be provided.")
    
    # Prepare the request
    headers = {
        "Content-Type": "application/json"
    }

    # Extract the accepted queries
    queries = {k: v for k, v in query.items() if k in accepted_queries}
    queries.update({k: v for k, v in kwargs.items() if k in accepted_queries})

    # Execute the request
    response = requests.request(
        method,
        url.format(**path, **kwargs),
        headers=headers,
        params=queries if len(queries) > 0 else None,
        json=body if len(body) > 0 else None
    )

    # Return the response (either as JSON or text)
    try:
        return response.json()
    except:
        return {"message": response.text}

In [ ]:
show_doc(generate_request)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/oas_to_requests.py#L289){target="_blank" style="float:right; font-size:smaller"}

### generate_request

>      generate_request (function_name:str, url:str, method:str, path:dict={},
>                        query:dict={}, body:dict={}, accepted_queries:list=[],
>                        **kwargs)

*Generate a request from the function name and parameters.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| function_name | str |  | The name of the function |
| url | str |  | The URL of the request |
| method | str |  | The method of the request |
| path | dict | {} | The path parameters of the request |
| query | dict | {} | The query parameters of the request |
| body | dict | {} | The body of the request |
| accepted_queries | list | [] | The accepted queries of the request |
| kwargs |  |  |  |
| **Returns** | **dict** |  | **The response of the request** |

In [ ]:
generate_request(
    function_name="weathercamStations",
    url="https://tie.digitraffic.fi/api/weathercam/v1/stations/{id}",
    method="GET",
    path={"id": "C01504"}
)

{'type': 'Feature',
 'id': 'C01504',
 'geometry': {'type': 'Point', 'coordinates': [24.235601, 60.536727, 0.0]},
 'properties': {'id': 'C01504',
  'name': 'vt2_Karkkila_Korpi',
  'cameraType': 'HIKVISION',
  'nearestWeatherStationId': 1052,
  'collectionStatus': 'GATHERING',
  'state': None,
  'dataUpdatedTime': '2024-12-09T15:27:13Z',
  'collectionInterval': 600,
  'names': {'fi': 'Tie 2 Karkkila, Kappeli',
   'sv': 'Väg 2 Högfors, Kappeli',
   'en': 'Road 2 Karkkila, Kappeli'},
  'roadAddress': {'roadNumber': 2,
   'roadSection': 13,
   'distanceFromRoadSectionStart': 3818,
   'carriageway': 'ONE_CARRIAGEWAY',
   'side': 'LEFT',
   'contractArea': '',
   'contractAreaCode': 344},
  'liviId': 'Livi1089298',
  'country': None,
  'startTime': '1995-06-01T00:00:00Z',
  'repairMaintenanceTime': None,
  'annualMaintenanceTime': None,
  'purpose': 'keli',
  'municipality': 'Karkkila',
  'municipalityCode': 224,
  'province': 'Uusimaa',
  'provinceCode': 1,
  'presets': [{'id': 'C0150401',
 

### API schema

Extract important information about the functions and creates a GPT-compatible toolbox schema. The idea is to convert all necessary information for generating an API request to a parameter for GPT to provide. As such, the parameters of each function in this toolbox schema will include:

- `url`: URL to send requests to (type `string`, with `const` default value formed with a base URL and endpoint path) 
- `method`: HTTP method for each endpoint (type `string`, with `const` value)
- `path`: dictionary for path parameters that maps parameter names to schema
- `query`: dictionary for query parameters that maps parameter names to schema
- `body`: request body schema

Test with usage from DigiTraffic:

In [ ]:
#| export
def api_schema(
        base_url: str,  # The base URL of the API
        oas: dict,  # The OpenAPI schema
        service_name: Optional[str]=None,  # The name of the service
        fixup:Callable=None, # a fixup function to execute a REST API when a function name isn't found.
    ) -> dict:  # The api schema
    """Form the toolbox schema from the OpenAPI schema."""
    
    # Extract the references
    flatten_refs = extract_refs(oas)
    
    # Initialize the toolbox
    toolbox = []

    # Traverse the paths section of the spec
    for path, methods in oas["paths"].items():
        for method, info in methods.items():
            # Extract the function name
            name = info["operationId"] if "operationId" in info else \
                f"{method}{path.replace('/', '_').replace('{', 'by').replace('}', '').replace('-', '_')}"
            name = re.sub(r'[^a-zA-Z0-9_-]', '_', name)

            # Extract the function description
            description = info["description"] if "description" in info else info.get("summary", "")

            # Extract the function parameters
            parameters = {
                "type": "object",
                "properties": {},   
                "required": []
            }
            accepted_queries = []

            # Extract endpoint parameters
            if "parameters" in info:
                for param in info["parameters"]:
                    # Extract the parameter location (query, path, header, cookie)
                    location = param.get("in", "query")

                    # Initialize the parameter object based on location
                    if location not in parameters["properties"]:
                        parameters["properties"][location] = {
                            "type": "object",
                            "properties": {},
                            "required": []
                        }

                    # Extract the parameter schema
                    param_obj, required = transform_property(param, flatten_refs)

                    # Add the parameter to the toolbox
                    parameters["properties"][location]["properties"][param["name"]] = param_obj
                    if required or location == "path":
                        parameters["properties"][location]["required"].append(param["name"])
                        parameters["required"].append(location)

                    # Add the parameter to the accepted queries
                    if location == "query":
                        accepted_queries.append(param["name"])
                    
            # Extract the function body
            body = {}
            if "requestBody" in info and "content" in info["requestBody"] \
                    and "application/json" in info["requestBody"]["content"] \
                    and "schema" in info["requestBody"]["content"]["application/json"]:
                body = info["requestBody"]["content"]["application/json"]
                body, _ = transform_property(body, flatten_refs)
                parameters["properties"]["body"] = body
                parameters["required"].append("body")

            # Remove duplicate required properties
            parameters["required"] = list(set(parameters["required"]))
                
            # Conclude the function information
            function = {
                "name": name,
                "description": description,
                "parameters": parameters,
                "metadata": {
                    "url": base_url + path,
                    "method": method,
                    "accepted_queries": accepted_queries,
                }
            }
            if fixup: function['fixup'] = f"{fixup.__module__}.{fixup.__name__}"
            if service_name: function["metadata"]["service"] = service_name

            # Add the function to the toolbox
            toolbox.append(
                {
                    "type": "function",
                    "function": function
                }
            )
        
    return toolbox

In [ ]:
show_doc(api_schema)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/oas_to_requests.py#L329){target="_blank" style="float:right; font-size:smaller"}

### api_schema

>      api_schema (base_url:str, oas:dict, service_name:Optional[str]=None,
>                  fixup:Callable=None)

*Form the toolbox schema from the OpenAPI schema.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| base_url | str |  | The base URL of the API |
| oas | dict |  | The OpenAPI schema |
| service_name | Optional | None | The name of the service |
| fixup | Callable | None | a fixup function to execute a REST API when a function name isn't found. |
| **Returns** | **dict** |  | **The api schema** |

In [ ]:
tool_schema = api_schema(base_url=BASE_URL, oas=oas, fixup=generate_request)
#pprint(tool_schema[:3])
for ts in tool_schema:
    if ts['function']['name']=='getLatestTrafficRestrictionNotificationById':
        print(json.dumps(ts['function'], indent=4))

### GPT integration

Integrate with existing `complete` function:

In [ ]:
#| eval: false
from llmcam.core.fn_to_fc import complete, form_msgs, print_msgs

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "Get the weather camera information for the stations with ID C01503 and C01504."),
])
complete(messages, tool_schema)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Get the weather camera information for the stations with ID C01503 and C01504.
>> Assistant:
Here is the weather camera information for the stations with IDs C01503 and C01504:  ### Station
C01503 (Road 51 Inkoo) - **Camera Type:** BOSCH - **Nearest Weather Station ID:** 1013 -
**Collection Status:** GATHERING - **Data Updated Time:** 2024-12-09T15:25:44Z - **Collection
Interval:** 600 seconds - **Municipality:** Inkoo - **Municipality Code:** 149 - **Province:**
Uusimaa - **Province Code:** 1  **Coordinates:** [Longitude: 23.99616, Latitude: 60.05374]
**Presets:** 1. **Inkooseen:**     - [Image](https://weathercam.digitraffic.fi/C0150301.jpg)    -
Direction: INCREASING_DIRECTION    - Resolution: 1280x720  2. **Hankoon:**     -
[Image](https://weathercam.digitraffic.fi/C0150302.jpg)    - Direction: DECREASING_DIRECTION    -
Resolution: 1280x720  3. **Tienpinta:**     -
[Image](https://w

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()